<a href="https://colab.research.google.com/github/Natkitta252603/Python_dataviz2024/blob/main/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd

In [21]:
import os
path = '/content/drive/MyDrive/data viz'
data_file_path = os.path.join(path, '/content/drive/MyDrive/data viz/test-TidyData_2024.xlsx')
print(data_file_path)

/content/drive/MyDrive/data viz/test-TidyData_2024.xlsx


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
df_Tidy = pd.read_excel(data_file_path)
df_Tidy.head()

,month,station,rep.,Chlo_a,Chlo_b,total_Caro,Temp.,pH,EC,Salinity,TDS,DO,Nitrate,Ammonium,Phosphate,No.Algal_cell,@dropdown
0,July_2018,1,1,0.094645,0.011954,0.026780,30.2,5.54,622.7,330.3,545.2,4.37,0.172902,0.052817,0.012748,1.635833e+05,NaN
1,July_2018,2,1,0.038433,0.059019,0.021387,29.1,6.24,580.7,307.0,488.2,6.55,0.154197,0.144843,0.006086,1.166667e+06,NaN
2,July_2018,3,1,0.043884,0.089415,0.023223,29.7,5.52,547.1,289.4,462.2,5.67,0.153097,0.154321,0.014158,2.970000e+06,NaN
3,July_2018,4,1,0.039836,0.064552,0.016537,29.2,5.92,473.6,249.2,397.9,0.69,0.157026,0.387408,0.056951,5.066667e+05,NaN
4,July_2018,5,1,0.044712,0.062424,0.022990,29.5,5.79,685.5,366.0,581.2,4.46,0.204024,0.043387,0.007239,1.265000e+06,NaN


#1. ความแตกต่างของ ค่าเฉลี่ย ของข้อมูล Chlo_a ทุกๆ station ระหว่างปี 2018 และ ปี 2019

In [24]:
#กรองข้อมูลปี2018 และ 2019
df_2018 = df_Tidy [df_Tidy ['month'].str.contains('2018')]
df_2019 = df_Tidy [df_Tidy ['month'].str.contains('2019')]

In [25]:
# คำนวณค่าเฉลี่ยของ Chlo_a สำหรับแต่ละสถานีในปี 2018
mean_2018 = df_2018.groupby('station')['Chlo_a'].mean().reset_index()
mean_2018.rename(columns={'Chlo_a': 'Mean_Chlo_a_2018'}, inplace=True)


In [26]:
# คำนวณค่าเฉลี่ยของ Chlo_a สำหรับแต่ละสถานีในปี 2019
mean_2019 = df_2019.groupby('station')['Chlo_a'].mean().reset_index()
mean_2019.rename(columns={'Chlo_a': 'Mean_Chlo_a_2019'}, inplace=True)


In [29]:
# รวมข้อมูลทั้งสองปีเข้าด้วยกัน
comparison = pd.merge(mean_2018, mean_2019, on='station')

In [30]:
# prompt: คำนวณความแตกต่างของค่าเฉลี่ย

# คำนวณความแตกต่างของค่าเฉลี่ย
comparison['Difference'] = comparison['Mean_Chlo_a_2019'] - comparison['Mean_Chlo_a_2018']

# แสดงผลลัพธ์
print(comparison)


   station  Mean_Chlo_a_2018  Mean_Chlo_a_2019  Difference
0        1          0.052529          0.018268   -0.034261
1        2          0.048112          0.027911   -0.020201
2        3          0.053285          0.023057   -0.030228
3        4          0.046794          0.021620   -0.025173
4        5          0.043432          0.020100   -0.023331
5        6          0.043809          0.015594   -0.028215


#2 สร้างตารางค่าเฉลี่ย Chlo_b ทุกๆ station ของแต่ละเดือน

In [11]:
# Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Load the Excel file
df_Tidy = pd.read_excel('/content/drive/MyDrive/data viz/test-TidyData_2024.xlsx')

# Check if 'date' column exists before processing
if 'date' in df_Tidy.columns:
    # แปลงคอลัมน์ 'date' ให้เป็น datetime
    df_Tidy['date'] = pd.to_datetime(df_Tidy['date'])

    # สร้างคอลัมน์ 'month' จาก 'date'
    df_Tidy['month'] = df_Tidy['date'].dt.to_period('M')

else:
    print("Warning: 'date' column not found in the DataFrame. Skipping date processing.")

# คำนวณค่าเฉลี่ยของ Chlo_b สำหรับแต่ละสถานีในแต่ละเดือน
average_chlo_b = df_Tidy.groupby(['station', 'month'])['Chlo_b'].mean().reset_index()

# สร้าง Pivot Table เพื่อจัดรูปแบบข้อมูล
pivot_table = average_chlo_b.pivot(index='month', columns='station', values='Chlo_b')

# แบ่ง station เป็น 6 columns
num_stations = len(pivot_table.columns)
stations_per_column = (num_stations + 5) // 6  # ปัดเศษขึ้นเพื่อให้แน่ใจว่าทุก station ถูกแสดง

result_tables = []
for i in range(6):
    start_index = i * stations_per_column
    end_index = min((i + 1) * stations_per_column, num_stations)
    subset_table = pivot_table.iloc[:, start_index:end_index]
    result_tables.append(subset_table)

# แสดงผลลัพธ์
for table in result_tables:
    display(table)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


station,1
month,
Apr_2019,0.051381
Aug_2018,0.075665
Dec_2018,0.070061
Jan_2019,0.022451
July_2018,0.046688
July_2019,0.014365
June_2019,0.020548
May_2019,0.015949
Nov_2018,0.075937


station,2
month,
Apr_2019,0.019378
Aug_2018,0.065403
Dec_2018,0.064375
Jan_2019,0.014968
July_2018,0.062731
July_2019,0.013691
June_2019,0.013194
May_2019,0.028765
Nov_2018,0.064552


station,3
month,
Apr_2019,0.030775
Aug_2018,0.065179
Dec_2018,0.080867
Jan_2019,0.017722
July_2018,0.073561
July_2019,0.017439
June_2019,0.013301
May_2019,0.015890
Nov_2018,0.097963


station,4
month,
Apr_2019,0.020170
Aug_2018,0.061289
Dec_2018,0.061951
Jan_2019,0.012142
July_2018,0.061395
July_2019,0.009257
June_2019,0.013821
May_2019,0.015559
Nov_2018,0.070605


station,5
month,
Apr_2019,0.027015
Aug_2018,0.060107
Dec_2018,0.070085
Jan_2019,0.009789
July_2018,0.064623
July_2019,0.011054
June_2019,0.013880
May_2019,0.016623
Nov_2018,0.063866


station,6
month,
Apr_2019,0.025655
Aug_2018,0.063027
Dec_2018,0.065663
Jan_2019,0.013856
July_2018,0.056193
July_2019,0.010073
June_2019,0.008583
May_2019,0.023882
Nov_2018,0.073360
